In [ ]:
import pickle as pkl
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from sklearn.utils import shuffle
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
savepath = './results/'
trainin = pkl.load(open(savepath + "trainin", "rb"))
trainout = pkl.load(open(savepath + "trainout", "rb"))
testin = pkl.load(open(savepath + "testin", "rb"))
testout = pkl.load(open(savepath + "testout", "rb"))

In [ ]:
print(trainin.shape)
print(trainout.shape)

In [ ]:
def mae(y_true, y_pred):
    y_pred = tf.cast(y_pred, tf.float32)
    loss = tf.keras.losses.mean_absolute_error(y_true, y_pred)
    return K.mean(loss)

In [ ]:
def mse(y_true, y_pred):
    y_pred = tf.cast(y_pred, tf.float32)
    loss = tf.keras.losses.MSE(y_true, y_pred)
    return K.mean(loss)

In [ ]:
#Input and output sizes
input_size = tf.transpose(trainin).shape[0]
output_size = tf.transpose(trainout).shape[0]

#ML Hyperparameters
lr = 10e-4 #learning rate
epochs = 100 #number of passes of the data
batchsize = 256 #number of images fed in at a time
iterations = trainin.shape[0]//batchsize #number of batches used per epoch
optimizer = tf.keras.optimizers.Adam(lr, beta_1=.9, beta_2=.999, epsilon=1e-4, decay=0.) #ML optimizer
loss_fn = mae #loss function

metric =  keras.metrics.MeanSquaredError() #metric to be used for training
val_metric = keras.metrics.MeanSquaredError() #metric to be used for validation

#Configuration dictionary for ML model
config = {'act1': 'relu', 'act2': 'linear', 'size1': 256, 'size2': 48, 'size3':12, 'size4': 3}

#ML model in keras functional API (CNN with max pooling fed into dense with batch normalization) 
inputs = keras.Input(shape = (input_size,), name = 'input')
x = Dense(int(config['size1']), input_shape = (input_size,), activation = config['act1'])(inputs)
x = Reshape(target_shape = (8, 8, 4))(x)
x = BatchNormalization()(x)
x = Conv2D(16, (3,3), activation = config['act1'], padding = "same")(x)
x = Conv2D(16, (3,3), activation = config['act1'], padding = "same")(x)
x = MaxPooling2D((2, 2), strides = 2)(x)
x = Conv2D(32, (3,3), activation = config['act1'], padding = "same")(x)
x = Conv2D(32, (3,3), activation = config['act1'], padding = "same")(x)
x = MaxPooling2D((2, 2), strides = 2)(x)
x = Conv2D(64, (3,3), activation = config['act1'], padding = "same")(x)
x = Conv2D(64, (3,3), activation = config['act1'], padding = "same")(x)
x = MaxPooling2D((2, 2), strides = 2)(x)
x = Flatten()(x)
x = Dense(int(config['size2']), activation = config['act1'])(x)
x = BatchNormalization()(x)
x = Dense(int(config['size3']), activation = config['act1'])(x)
x = BatchNormalization()(x)
x = Dense(int(config['size4']), activation = config['act1'])(x)
x = BatchNormalization()(x)
main_output = Dense(output_size, activation = config['act2'], name = 'main_output')(x)

outputs = [main_output]

model = keras.Model(inputs = inputs, outputs = outputs)
modelname = ("BnCNN_%.1e_%d_%d" % (lr, epochs, batchsize))

In [ ]:
#index list to be used for shuffling
indices = tf.range(start = 0, limit = tf.shape(trainin)[0], dtype=tf.int32)

In [ ]:
# #Function that trains the model on a batch of declared size
# def train_model_on_batch():
#     #Use index list shuffling to create shuffled batches for training
#     start = iteration * batchsize
#     x_batch = shuffledin[start:start + batchsize, :]
#     y_batch = shuffledout[start:start + batchsize, :]
    
#     #Gradient tape watches changes to loss as model is called
#     with tf.GradientTape() as tape:
#         current_loss = loss_fn(y_batch, model(x_batch))
    
#     #Apply gradients to optimizer based on shape of model
#     gradients = tape.gradient(current_loss, model.trainable_variables)
#     optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
#     #Update the training metric
#     metric.update_state(y_batch, model(x_batch))
    
    
#     #Only calculate validation metric and validation loss once
#     if iteration == (iterations - 1):
#         val_metric.update_state(testout, model(testin))
#         val_loss = loss_fn(testout, model(testin))
#         return current_loss, metric.result(), val_loss, val_metric.result()
#     else:
#         return current_loss, 0, 0, 0

In [ ]:
# #Training history lists
# loss_history = []
# metric_history = []
# val_loss_history = []
# val_metric_history = []

# #Initial loss value below which the model will start saving
# min_loss = 1

# #Customizable model training loop
# for epoch in range(epochs):
#     shuffled_indices = tf.random.shuffle(indices)
#     shuffledin = tf.gather(trainin, shuffled_indices)
#     shuffledout = tf.gather(trainout, shuffled_indices)
#     for iteration in range(iterations):
#         print("Iteration: %d / %d" % (iteration, iterations))
#         current_loss, metric_result, val_loss, val_metric_result = train_model_on_batch()
#     #For runs with many epochs, control how often loss histories are recorded and printed
#     if epoch % 1 == 0:
#         loss_history.append(current_loss.numpy())
#         metric_history.append(metric_result.numpy())
#         val_loss_history.append(val_loss.numpy())
#         val_metric_history.append(val_metric_result.numpy())
#         print("\nEpoch: {}/{} - Loss: {} - MSE: {} \n\nVal_Loss: {} - Val_MSE: {}".format(
#             (epoch + 1), epochs, loss_history[-1], metric_history[-1], val_loss_history[-1], val_metric_history[-1]))
#     #Save the model as h5 filetype which is smaller than full model data
#     if val_metric_result < .3:
#         if val_metric_result < min_loss:
#             model.save("%s\\%s.h5" % (resDir,modelname))
#             min_loss = val_metric_result
#     #Reset states of metrics
#     metric.reset_states()
#     val_metric.reset_states()

In [ ]:
model.compile(optimizer = optimizer, loss = 'mae', metrics = 'mse')

In [ ]:
model.fit(trainin, trainout, batch_size = batchsize, epochs = epochs, verbose = 2, validation_data = (testin, testout))

In [ ]:
model.save("./results/%s.h5" % (modelname))